In [ ]:
import pandas as pd
import json

# Assuming we have the following data
data = {
    "transaction_id": ["T1001", "T1002", "T1003", "T1004", "T1005"],
    "customer_id": ["C001", "C002", "C003", "C002", "C001"],
    "payment_amount": [125.50, 89.99, 120.00, 54.30, 210.20],
    "payment_date": [
        "2021-10-05",
        "2021-10-06",
        "2021-10-07",
        "2021-10-05",
        "2021-10-08",
    ],
    "payment_status": ["Paid", "Unpaid", "Paid", "Paid", "Pending"],
}

# Create DataFrame
df = pd.DataFrame(data)


company_names = {
    "ucb": "UCB.BR",
    "abinbev": "ABI.BR",
    "aedifica": "AED.BR",
    "ageas": "AGS.BR",
    "aperam": "APAM.AS",
    "argenx": "ARGX.BR",
    "belgian national bank": "BNB.BR",
    "cofinimmo": "COFB.BR",
    "elia": "ELI.BR",
    "ishares msci belgium etf": "EWK",
    "groupe bruxelles lambert sa": "GBLB.BR",
    "galapagos nv": "GLPG.AS",
    "kbc bank": "KBC.BR",
    "melexis nv": "MELE.BR",
    "proximus": "PROX.BR",
    "sofina": "SOF.BR",
    "solvay": "SOLB.BR",
    "umicore": "UMI.BR",
    "unified post group": "UPG.BR",
    "warehouses de pauw": "WDP.BR",
    "xior student housing": "XIOR.BR",
}


def retrieve_payment_status(df: data, transaction_id: str) -> str:
    if transaction_id in df.transaction_id.values:
        return json.dumps(
            {"status": df[df.transaction_id == transaction_id].payment_status.item()}
        )
    return json.dumps({"error": "transaction id not found."})


def retrieve_payment_date(df: data, transaction_id: str) -> str:
    if transaction_id in df.transaction_id.values:
        return json.dumps(
            {"date": df[df.transaction_id == transaction_id].payment_date.item()}
        )
    return json.dumps({"error": "transaction id not found."})

In [7]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "retrieve_payment_status",
            "description": "Get payment status of a transaction",
            "parameters": {
                "type": "object",
                "properties": {
                    "transaction_id": {
                        "type": "string",
                        "description": "The transaction id.",
                    }
                },
                "required": ["transaction_id"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "retrieve_payment_date",
            "description": "Get payment date of a transaction",
            "parameters": {
                "type": "object",
                "properties": {
                    "transaction_id": {
                        "type": "string",
                        "description": "The transaction id.",
                    }
                },
                "required": ["transaction_id"],
            },
        },
    },
]

In [8]:
import functools

names_to_functions = {
    "retrieve_payment_status": functools.partial(retrieve_payment_status, df=df),
    "retrieve_payment_date": functools.partial(retrieve_payment_date, df=df),
}

In [9]:
messages = [{"role": "user", "content": "What's the status of my transaction T1001?"}]

In [10]:
def get_api_key(platform, file_path):
    with open(file_path, "r") as file:
        lines = file.readlines()

    for line in lines:
        if "=" in line:
            key_value = line.split("=")
            key_platform = key_value[0].strip()
            key = key_value[1].strip().strip('"')

            if key_platform == platform:
                return key
    return None


key = get_api_key("mistral", "/Users/naimsassine/Desktop/DSAI/keys")

In [13]:
import os
from mistralai import Mistral


api_key = key
model = "mistral-small-latest"

client = Mistral(api_key=api_key)
response = client.chat.complete(
    model=model,
    messages=messages,
    tools=tools,
    tool_choice="any",
)
response

ChatCompletionResponse(id='48c72a1f83744e38953ac90d71557589', object='chat.completion', model='mistral-small-latest', usage=UsageInfo(prompt_tokens=166, completion_tokens=30, total_tokens=196), created=1730227179, choices=[ChatCompletionChoice(index=0, message=AssistantMessage(content='', tool_calls=[ToolCall(function=FunctionCall(name='retrieve_payment_status', arguments='{"transaction_id": "T1001"}'), id='hTa9738o5', type='function')], prefix=False, role='assistant'), finish_reason='tool_calls')])

In [14]:
messages.append(response.choices[0].message)

In [16]:
import json

tool_call = response.choices[0].message.tool_calls[0]
function_name = tool_call.function.name
function_params = json.loads(tool_call.function.arguments)
print("\nfunction_name: ", function_name, "\nfunction_params: ", function_params)


function_name:  retrieve_payment_status 
function_params:  {'transaction_id': 'T1001'}


In [17]:
function_result = names_to_functions[function_name](**function_params)
function_result

'{"status": "Paid"}'

In [18]:
messages.append(
    {
        "role": "tool",
        "name": function_name,
        "content": function_result,
        "tool_call_id": tool_call.id,
    }
)

response = client.chat.complete(model=model, messages=messages)
response.choices[0].message.content

'The transaction T1001 has been paid.'